In [1]:
{-# LANGUAGE NegativeLiterals #-}
{-# LANGUAGE FlexibleContexts #-}
{-# LANGUAGE OverloadedStrings #-}
{-# LANGUAGE TypeFamilies #-}

In [2]:
-- import Prelude hiding ((++))
import Data.Text (Text)
import qualified Data.Text as T
import qualified Data.Text.IO as TIO

import Text.Megaparsec hiding (State)
import qualified Text.Megaparsec.Lexer as L
import Text.Megaparsec.Text (Parser)
import qualified Control.Applicative as CA

import qualified Data.MultiSet as B -- B for bag
import qualified Data.Set as S

import Data.Either

In [3]:
type Part = B.MultiSet Integer
type Parts = B.MultiSet Part
type Candidates = S.Set Part
data Bridge = Bridge { bridgeParts :: Parts, requiring :: Integer } deriving (Eq, Show, Ord)
type Bridges = S.Set Bridge

In [4]:
-- really persuade Megaparsec not to include newlines in how it consume spaces.
onlySpace = (char ' ') <|> (char '\t')

sc :: Parser ()
sc = L.space (skipSome onlySpace) CA.empty CA.empty

lexeme = L.lexeme sc
integer = lexeme L.integer
symbol = L.symbol sc
slash = symbol "/"

partsP = partP `sepBy` newline
partP = B.fromList <$> integer `sepBy` slash

successfulParse :: Text -> Parts
successfulParse input = 
        case parse partsP "input" input of
                Left  _error -> B.empty -- TIO.putStr $ T.pack $ parseErrorPretty err
                Right partsList -> B.fromList partsList

In [5]:
sample = T.pack "42/37\n28/28\n29/25"

In [6]:
parseTest partsP (T.pack "42/37\n29/25")

[fromOccurList [(37,1),(42,1)],fromOccurList [(25,1),(29,1)]]

In [7]:
sampleParts = successfulParse sample
sampleParts

fromOccurList [(fromOccurList [(25,1),(29,1)],1),(fromOccurList [(28,2)],1),(fromOccurList [(37,1),(42,1)],1)]

In [8]:
text <- TIO.readFile "../../data/advent24.txt"
parts = successfulParse text
B.size parts

57

In [9]:
emptyBridge :: Bridge
emptyBridge = Bridge { bridgeParts = B.empty, requiring = 0}

In [10]:
hasPort :: Part -> Integer -> Bool
hasPort part port = port `B.member` part

In [11]:
available :: Parts -> Part -> Bridge -> Bool
available parts part bridge = B.occur part parts > B.occur part (bridgeParts bridge)

In [12]:
candidates :: Parts -> Bridge -> Candidates
candidates parts bridge = B.toSet $ B.filter canUse parts
    where needed = requiring bridge
          canUse p = hasPort p needed && available parts p bridge

In [13]:
candidates parts emptyBridge

fromList [fromOccurList [(0,1),(7,1)],fromOccurList [(0,1),(22,1)],fromOccurList [(0,1),(35,1)]]

In [14]:
grow :: Bridge -> Part -> Bridge
grow bridge part = bridge {bridgeParts = bp', requiring = req'}
    where req = requiring bridge
          req' = B.findMin $ B.delete req part
          bp' = B.insert part $ bridgeParts bridge

In [15]:
S.map (grow emptyBridge) $ candidates parts emptyBridge

fromList [Bridge {bridgeParts = fromOccurList [(fromOccurList [(0,1),(7,1)],1)], requiring = 7},Bridge {bridgeParts = fromOccurList [(fromOccurList [(0,1),(22,1)],1)], requiring = 22},Bridge {bridgeParts = fromOccurList [(fromOccurList [(0,1),(35,1)],1)], requiring = 35}]

In [16]:
extendOneBridge :: Parts -> Bridge -> Either Bridge Bridges
extendOneBridge parts bridge = 
    if S.null $ candidates parts bridge
    then Left bridge
    else Right (S.map (grow bridge) $ candidates parts bridge)
                                

In [17]:
extendBridges :: Parts -> Bridges -> Bridges -> Bridges
extendBridges parts bridges completed = 
    if S.null bridges then completed
                      else extendBridges parts bridges' completed'
    where updates = map (extendOneBridge parts) $ S.toList bridges
          newCompleted = lefts updates
          completed' = S.union completed $ S.fromList newCompleted
          bridges' = S.unions $ rights updates
          

In [18]:
allBridges parts = extendBridges parts (S.singleton emptyBridge) S.empty

In [19]:
S.size $ allBridges parts

49096

In [22]:
bridgeStrength :: Bridge -> Integer
bridgeStrength bridge = B.fold (+) 0 $ B.map partStrength $ bridgeParts bridge
    where partStrength = sum . B.elems 

In [25]:
S.findMax $ S.map bridgeStrength $ allBridges parts

1940

In [27]:
bridgeLength :: Bridge -> Int
bridgeLength bridge = B.size $ bridgeParts bridge

In [28]:
bestBridge parts = S.findMax $ S.map bridgeStrength longBridges
    where bridges = allBridges parts
          longest = S.findMax $ S.map bridgeLength bridges
          longBridges = S.filter (\b -> bridgeLength b == longest) bridges
          

In [29]:
bestBridge parts

1928